Example: Generating data
========================

Motivation
----------

Given the fundamental problem of Causal Inference, simulating or
generating data is of particular relevance when working with ATE or
CATE estimation: it allows to have a ground truth that we don't get
from the real world.

For instance, when generating data, we can have access to the
Individual Treatment Effect and use that ground truth to evaluate a
treatment effect method at hand.

In the following example we will describe how the modules
{mod}`metalearners.data_generation` and
{mod}`metalearners.outcome_functions` can be used to generate data in
light of treatment effect estimation.


How-to
------

In the context of treatment effect estimation, our data usually
consists of 3 ingredients:

- Covariates
- Treatment assignments
- Observed outcomes

In this particular scenario of simulating data, we can add some
quantities of interest which are not available in the real world:

- Potential outcomes
- True CATE or true ITE

Let's generate those quantities one after another.


### Covariates

Let's start by generating covariates. We will use
{func}`metalearners.data_generation.generate_covariates` for that
purpose.

In [ ]:
from metalearners.data_generation import generate_covariates

features, categorical_features_idx, n_categories = generate_covariates(
        n_obs=1000,
        n_features=8,
        n_categoricals=3,
        format="pandas",
)
features.head() # type: ignore

We see that we generated a DataFrame with 8 columns of which the last
three are categoricals.


### Treatment assignments

In this example we will replicate the setup of an RCT, i.e. where the
treatment assignments are independent of the covariates. We rely on
{func}`metalearners.data_generation.generate_treatment`.

In [ ]:
import numpy as np
from metalearners.data_generation import generate_treatment

# We use a fair conflip as a reference.
propensity_scores = .5 * np.ones(1000)
treatment = generate_treatment(propensity_scores)
type(treatment), np.unique(treatment), treatment.mean()

As we would expect, an array of binary assignments is generated. The
average approximately corresponds to the universal propensity score of
.5.


### Potential outcomes

In this example we will rely on
{func}`metalearners.outcome_functions.linear_treatment_effect`, which
generates additive treatment effects which are linear in the features.
Note that there are other potential outcome functions available.

In [ ]:
from metalearners._utils import get_linear_dimension
from metalearners.outcome_functions import linear_treatment_effect

dim = get_linear_dimension(features)
outcome_function = linear_treatment_effect(dim)
potential_outcomes = outcome_function(features)
potential_outcomes

We see it generates one column with the potential outcome {math}`Y(0)` and one column
with the potential outcome {math}`Y(1)`. The individual treatment
effect can be inferred as a subtraction of both.

### Observed outcomes

Lastly, we can combine the treatment assignments and potential
outcomes to generate the observed outcomes. Note that there might be
noise which distinguishes the potential outcome from the observed
outcome. For that purpose we can use
{func}`metalearners.data_generation.compute_experiment_outputs` and run

In [ ]:
from metalearners.data_generation import compute_experiment_outputs

observed_outcomes, true_cate = compute_experiment_outputs(
    potential_outcomes,
    treatment,
)